In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import skimage.io as io
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

In [2]:
# Check NumPy version and confirm fix
import numpy as np
print(f"NumPy version: {np.__version__}")

# Fix applied: Rebuilt the Cython extension with current NumPy version
print("✓ Fixed: Rebuilt Cython extension for NumPy 2.x compatibility")
print("The pycocotools._mask module should now work with NumPy 2.x")

# Test import to verify the fix
try:
    from pycocotools import _mask as maskUtils
    print("✓ Successfully imported pycocotools._mask")
except Exception as e:
    print(f"✗ Import failed: {e}")

NumPy version: 2.3.2
✓ Fixed: Rebuilt Cython extension for NumPy 2.x compatibility
The pycocotools._mask module should now work with NumPy 2.x
✓ Successfully imported pycocotools._mask


In [3]:
annType = ['segm','bbox','keypoints']
annType = annType[0]      #specify type here
prefix = 'person_keypoints' if annType=='keypoints' else 'instances'
print('Running demo for *%s* results.'%(annType))

Running demo for *segm* results.


In [4]:
#initialize COCO ground truth api
dataDir='../'
dataType='val2014'
annFile = 'data/instances_val2014.json'
cocoGt=COCO(annFile)

loading annotations into memory...
Done (t=2.14s)
creating index...
index created!


In [ ]:
# Initialize COCO detections API
# Construct path to results file using string formatting:
# - dataDir: parent directory ('../')
# - prefix: 'instances' for segm/bbox, 'person_keypoints' for keypoints
# - dataType: 'val2014' (validation set from 2014)
# - annType: annotation type ('segm', 'bbox', or 'keypoints')
# Example result: '../results/instances_val2014_fakesegm100_results.json'
resFile='%s/results/%s_%s_fake%s100_results.json'
resFile = resFile%(dataDir, prefix, dataType, annType)

# Load the detection results into a COCO object for evaluation
# This processes the results file and creates a COCO API object compatible with evaluation
cocoDt=cocoGt.loadRes(resFile)

Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!


# SHOWCASING DATAFILES

In [6]:
import json
annotation_data = json.load(open(annFile, 'r'))
res_data = json.load(open(resFile, 'r'))

In [7]:
print(annotation_data.keys())

dict_keys(['info', 'images', 'licenses', 'annotations', 'categories'])


In [8]:
print(annotation_data['info'])

{'description': 'COCO 2014 Dataset', 'url': 'http://cocodataset.org', 'version': '1.0', 'year': 2014, 'contributor': 'COCO Consortium', 'date_created': '2017/09/01'}


In [9]:
type(annotation_data['images'])

list

In [10]:
annotation_data['images'][0]

{'license': 3,
 'file_name': 'COCO_val2014_000000391895.jpg',
 'coco_url': 'http://images.cocodataset.org/val2014/COCO_val2014_000000391895.jpg',
 'height': 360,
 'width': 640,
 'date_captured': '2013-11-14 11:18:45',
 'flickr_url': 'http://farm9.staticflickr.com/8186/8119368305_4e622c8349_z.jpg',
 'id': 391895}

In [11]:
type(annotation_data['annotations'])

list

In [12]:
annotation_data['annotations'][0]

{'segmentation': [[239.97,
   260.24,
   222.04,
   270.49,
   199.84,
   253.41,
   213.5,
   227.79,
   259.62,
   200.46,
   274.13,
   202.17,
   277.55,
   210.71,
   249.37,
   253.41,
   237.41,
   264.51,
   242.54,
   261.95,
   228.87,
   271.34]],
 'area': 2765.1486500000005,
 'iscrowd': 0,
 'image_id': 558840,
 'bbox': [199.84, 200.46, 77.71, 70.88],
 'category_id': 58,
 'id': 156}

In [13]:
res_data[0]

{'image_id': 42,
 'category_id': 18,
 'segmentation': {'size': [478, 640],
  'counts': 'VQi31m>0O2N100O100O2N100O10001N101O1N2O1O2M2O1O1N3N1O1N2O2N1N2O1O1N3N1O1N2O2N1N2O1O2M2O1O1M3M4K4M3M3M4L3M3M3M4L3L4M3M3M4L3M3M3M4L3O1N2N101N1O2O0O2N101N1O2O0O2N101N1O2O0O1O2N101N1O2O0O2N101N1O2O0O2N101N1O2O0O1O2O0O2N101N1O2O0O2N101N101O001O1O001O1N2O001O1O1O001O1O1O001O1O001O1O1N101O1O1O001O1O1O001O1O1O001O1N2O001O1O001O1O1O001O1O1O001O1O1N010000O10000O10000O10000O100O010O100O100O100O10000O100O100O10O0100O100O100O100O1O100O100O1O010O100O1O2O0O2N101N101N1O2O1N1O2O0O2O0O2N2O0O2N101N101N2N101N101N1O2O1N1O2O0O20O2O0O2O001N101N100O2O001N101N101N101O0O101N101N101N101O0O101N101N1010O010O010O00010O0O2N1O2N1O2N1O2N1O2N1O2N1O2N1O2N1O2N1O2N1O2N1O2N1O2N1O2N1O2N1O2M2M4L3M4L3M4RNREGP;5UEGo:3XEHk:4ZEHj:2\\EJg:1_EKe:0`ELc:OcEMa:NdEN_:MgE0\\:JjE2Y:JlE2X:HnE4a<LZd?'},
 'score': 0.236}

# CONTINUING ON WITH COCO DETECTIONS

Results file is loaded also into coco class to get detections.

In [14]:
print(f"cocoGt type: {type(cocoGt)}")
print(f"cocoDt type: {type(cocoDt)}")

cocoGt type: <class 'pycocotools.coco.COCO'>
cocoDt type: <class 'pycocotools.coco.COCO'>


In [15]:
imgIds=sorted(cocoGt.getImgIds())
imgIds=imgIds[0:100]
imgId = imgIds[np.random.randint(100)]

In [ ]:
# running evaluation
print("annType: %s"%(annType))
cocoEval = COCOeval(cocoGt,cocoDt,annType)
cocoEval.params.imgIds  = imgIds
cocoEval.params.useCats = 0
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

annType: segm
Running per image evaluation...
Evaluate annotation type *segm*
